In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "knn7u"
fs_method, fs_ratio = "fcls", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9146 | AUPR: 0.6518 | Acc: 95.29%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9184 | AUPR: 0.6614 | Acc: 95.31%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9209 | AUPR: 0.6693 | Acc: 95.36%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9148 | AUPR: 0.6540 | Acc: 95.32%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9189 | AUPR: 0.6652 | Acc: 95.35%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9199 | AUPR: 0.6690 | Acc: 95.45%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9209 | AUPR: 0.6693 | Acc: 95.36%
Internal test: AUC: 0.9279 | AUPR: 0.7070 | Acc: 95.35%
External test: AUC: 0.9228 | AUPR: 0.6679 | Acc: 95.10%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9155 | AUPR: 0.6258 | Acc: 95.02%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9159 | AUPR: 0.6198 | Acc: 94.92%
model(kernel=linear, C=1)
Valid:         AUC: 0.9152 | AUPR: 0.6123 | Acc: 94.82%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9159 | AUPR: 0.6198 | Acc: 94.92%
Internal test: AUC: 0.9199 | AUPR: 0.6726 | Acc: 94.96%
External test: AUC: 0.9159 | AUPR: 0.6073 | Acc: 94.57%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7576 | AUPR: 0.4381 | Acc: 94.80%
model(n_neighbors=5)
Valid:         AUC: 0.7794 | AUPR: 0.4815 | Acc: 95.08%
model(n_neighbors=7)
Valid:         AUC: 0.7941 | AUPR: 0.4967 | Acc: 95.10%
best model(n_neighbors=7)
Valid:         AUC: 0.7941 | AUPR: 0.4967 | Acc: 95.10%
Internal test: AUC: 0.7867 | AUPR: 0.5096 | Acc: 94.89%
External test: AUC: 0.8030 | AUPR: 0.5070 | Acc: 94.90%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8722 | AUPR: 0.3985 | Acc: 86.21%
best model()
Valid:         AUC: 0.8722 | AUPR: 0.3985 | Acc: 86.21%
Internal test: AUC: 0.8823 | AUPR: 0.4280 | Acc: 86.25%
External test: AUC: 0.8700 | AUPR: 0.3803 | Acc: 84.63%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9169 | AUPR: 0.6492 | Acc: 95.23%
model(C=10.0)
Valid:         AUC: 0.9166 | AUPR: 0.6479 | Acc: 95.21%
model(C=100.0)
Valid:         AUC: 0.9163 | AUPR: 0.6475 | Acc: 95.19%
best model(C=1.0)
Valid:         AUC: 0.9169 | AUPR: 0.6492 | Acc: 95.23%
Internal test: AUC: 0.9228 | AUPR: 0.6906 | Acc: 95.26%
External test: AUC: 0.9155 | AUPR: 0.6325 | Acc: 94.86%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8623 | AUPR: 0.5567 | Acc: 86.91%
model(criterion=log_loss)
Valid:         AUC: 0.8665 | AUPR: 0.5708 | Acc: 88.31%
model(criterion=entropy)
Valid:         AUC: 0.8665 | AUPR: 0.5708 | Acc: 88.31%
best model(criterion=log_loss)
Valid:         AUC: 0.8665 | AUPR: 0.5708 | Acc: 88.31%
Internal test: AUC: 0.8764 | AUPR: 0.6079 | Acc: 89.56%
External test: AUC: 0.8726 | AUPR: 0.5835 | Acc: 88.82%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9114 | AUPR: 0.6497 | Acc: 89.05%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9129 | AUPR: 0.6518 | Acc: 88.99%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9128 | AUPR: 0.6528 | Acc: 89.07%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9114 | AUPR: 0.6526 | Acc: 89.01%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9125 | AUPR: 0.6540 | Acc: 89.27%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9125 | AUPR: 0.6551 | Acc: 89.43%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9114 | AUPR: 0.6526 | Acc: 89.01%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9125 | AUPR: 0.6540 | Acc: 89.27%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9125 | AUPR: 0.6551 | Acc: 89.43%
best model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9129 | AUPR: 0.6518 | Acc: 8

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9193 | AUPR: 0.6554 | Acc: 95.18%
model(criterion=squared_error)
Valid:         AUC: 0.9194 | AUPR: 0.6583 | Acc: 95.19%
best model(criterion=squared_error)
Valid:         AUC: 0.9194 | AUPR: 0.6583 | Acc: 95.19%
Internal test: AUC: 0.9243 | AUPR: 0.6965 | Acc: 95.33%
External test: AUC: 0.9217 | AUPR: 0.6549 | Acc: 95.24%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9120 | AUPR: 0.6257 | Acc: 95.12%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9130 | AUPR: 0.6256 | Acc: 95.09%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9131 | AUPR: 0.6246 | Acc: 95.01%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9131 | AUPR: 0.6246 | Acc: 95.01%
Internal test: AUC: 0.9184 | AUPR: 0.6852 | Acc: 95.21%
External test: AUC: 0.9141 | AUPR: 0.6190 | Acc: 94.62%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)